In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tqdm import tqdm

import warnings
from tensorflow import get_logger

warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', UserWarning)
get_logger().setLevel('ERROR')

# Loading the dataset
df = pd.read_csv("Microsoft_Stock.csv")
close_series = df["Close"]

########################### Single Split ###########################

# Train-Test 80-20 split
train_size = round(0.8 * len(close_series))

# Specifying Train and Test data
train_data = close_series[:train_size].to_numpy().reshape(1, train_size, 1)
target_data = close_series[train_size:].to_numpy().reshape(1, len(close_series) - train_size, 1)

# Training LSTM on the train data
lstm_model = Sequential()
lstm_model.add(LSTM(units=8, input_shape=(train_size, 1)))
lstm_model.add(Dense(len(close_series) - train_size))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

# Predicting the future prices, using trained LSTM model
lstm_model.fit(train_data, target_data, epochs=2, batch_size=1, verbose=0)
pred_lstm = lstm_model.predict(train_data, verbose=0)

# Calculating the Error of LSTM's prediction
pred_err_lstm = mean_squared_error(target_data.ravel(), pred_lstm.ravel())

# Calculating the average Error of LSTM's prediction
print(f"LSTM's prediction error (MSE) for 80-20 Single Split is: {pred_err_lstm}")

########################### Sliding Window ###########################

# Sliding window strategy parameters
train_size_list = [7, 30] # Replave with [7, 30, 100, 30, 100] for full evaluation
pred_size_list = [1, 1] # Replave with [1, 1, 1, 7, 7] for full evaluation
slide_size_list = [1, 1] # Replave with [1, 1, 1, 7, 7] for full evaluation


# The fine-tuning loop
for train_size, pred_size, slide_size in zip(train_size_list, pred_size_list, slide_size_list):
    # Error list
    error_list = []

    # Looping over the data using sliding window
    for i in tqdm(range(train_size, len(close_series) - pred_size + 1, slide_size)):
        # Specifying Train and Test data
        train_data = close_series[(i-train_size):i].to_numpy().reshape(1, train_size, 1)
        target_data = close_series[i:(i+pred_size)].to_numpy().reshape(1, pred_size, 1)

        # Training LSTM on the train data
        lstm_model = Sequential()
        lstm_model.add(LSTM(units=8, input_shape=(train_size, 1)))
        lstm_model.add(Dense(pred_size))
        lstm_model.compile(loss='mean_squared_error', optimizer='adam')

        # Predicting the future prices, using trained LSTM model
        lstm_model.fit(train_data, target_data, epochs=2, batch_size=1, verbose=0)
        pred_lstm = lstm_model.predict(train_data, verbose=0)

        # Calculating the Error of LSTM's prediction
        pred_err_lstm = mean_squared_error(target_data.ravel(), pred_lstm.ravel())
        error_list.append(pred_err_lstm)

    # Calculating the average Error of LSTM's prediction
    print(f"Average LSTM's prediction error (MSE) for Sliding Window with size \
    {train_size}, Prediction Size {pred_size}, and Slide Size {slide_size}: {np.mean(error_list)}")

LSTM's prediction error (MSE) for 80-20 Single Split is: 41499.20639602444


100%|██████████| 1504/1504 [32:39<00:00,  1.30s/it]


Average LSTM's prediction error (MSE) for Sliding Window with size     7, Prediction Size 1, and Slide Size 1: 14807.435641128235


100%|██████████| 1481/1481 [34:48<00:00,  1.41s/it]

Average LSTM's prediction error (MSE) for Sliding Window with size     30, Prediction Size 1, and Slide Size 1: 15005.443116420838


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tqdm import tqdm

import warnings
from tensorflow import get_logger

warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', UserWarning)
get_logger().setLevel('ERROR')

# Loading the dataset
df = pd.read_csv("Microsoft_Stock.csv")
close_series = df["Close"]


# ########################### Expanding Window ###########################

# Expanding window strategy parameters
train_size_list = [30, 100]
pred_size_list = [1, 1]
slide_size_list = [1, 1]


# The fine-tuning loop
for train_size, pred_size, slide_size in zip(train_size_list, pred_size_list, slide_size_list):
    # Error list
    error_list = []

    # Looping over the data using expanding window
    for i in tqdm(range(train_size, len(close_series) - pred_size + 1, slide_size)):
        # Specifying Train and Test data
        train_data = close_series[:i].to_numpy().reshape(1, i, 1)
        target_data = close_series[i:(i+pred_size)].to_numpy().reshape(1, pred_size, 1)

        # Training LSTM on the train data
        lstm_model = Sequential()
        lstm_model.add(LSTM(units=8, input_shape=(i, 1)))
        lstm_model.add(Dense(pred_size))
        lstm_model.compile(loss='mean_squared_error', optimizer='adam')

        # Predicting the future prices, using trained LSTM model
        lstm_model.fit(train_data, target_data, epochs=2, batch_size=1, verbose=0)
        pred_lstm = lstm_model.predict(train_data, verbose=0)

        # Calculating the Error of LSTM's prediction
        pred_err_lstm = mean_squared_error(target_data.ravel(), pred_lstm.ravel())
        error_list.append(pred_err_lstm)

    # Calculating the average Error of LSTM's prediction
    print(f"Average LSTM's prediction error (MSE) for Expanding Window with intitial size \
    {train_size}, Prediction Size {pred_size}, and Expand Size {slide_size}: {np.mean(error_list)}")

100%|██████████| 1481/1481 [37:47<00:00,  1.53s/it]


Average LSTM's prediction error (MSE) for Expanding Window with intitial size     30, Prediction Size 1, and Expand Size 1: 15014.981251249174


100%|██████████| 1411/1411 [37:10<00:00,  1.58s/it]

Average LSTM's prediction error (MSE) for Expanding Window with intitial size     100, Prediction Size 1, and Expand Size 1: 15647.249746989055
